## Preprocessing

In [77]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [78]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [79]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [80]:
# Look at APPLICATION_TYPE value counts for binning
value_counts = application_df['APPLICATION_TYPE'].value_counts()
value_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [81]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = value_counts[value_counts < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [82]:
# Look at CLASSIFICATION value counts for binning
value_counts = application_df['CLASSIFICATION'].value_counts()
value_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [83]:
# You may find it helpful to look at CLASSIFICATION value counts >1
value_counts[value_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [84]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = value_counts[value_counts < 100].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [85]:
drop_cols = ['SPECIAL_CONSIDERATIONS']
application_df = application_df.drop(drop_cols, axis=1)

In [86]:
# Convert categorical data to numeric with `pd.get_dummies`
categoricals = ['APPLICATION_TYPE', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'CLASSIFICATION']
categoricals = [c for c in categoricals if c not in drop_cols]
application_join_df = application_df.join(pd.get_dummies(application_df[categoricals], dtype=float))
application_join_df = application_join_df.drop(categoricals, axis=1)


In [87]:
# Split our preprocessed data into our features and target arrays
X = application_join_df.drop(['IS_SUCCESSFUL'], axis=1).astype('float32').values
y = application_join_df['IS_SUCCESSFUL'].values[:, None].astype('float32')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [88]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [89]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inp_dim = X.shape[1]
hidden_dim = 2*inp_dim

nn = tf.keras.models.Sequential()
act = tf.keras.layers.ReLU()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_dim, input_dim=inp_dim))
nn.add(tf.keras.layers.LayerNormalization())
nn.add(tf.keras.layers.BatchNormalization())
nn.add(tf.keras.layers.Activation(act))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_dim//2))
nn.add(tf.keras.layers.LayerNormalization())
nn.add(tf.keras.layers.BatchNormalization())
nn.add(act)

# Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.hard_sigmoid))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 94)                4512      
                                                                 
 layer_normalization_3 (Lay  (None, 94)                188       
 erNormalization)                                                
                                                                 
 batch_normalization_3 (Bat  (None, 94)                376       
 chNormalization)                                                
                                                                 
 activation_4 (Activation)   (None, 94)                0         
                                                                 
 dense_15 (Dense)            (None, 47)                4465      
                                                                 
 layer_normalization_4 (Lay  (None, 47)               

In [90]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(1e-3), metrics=["accuracy"])

In [91]:
cb = [tf.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy",factor=0.1,patience=10,verbose=1),
      tf.keras.callbacks.ModelCheckpoint("model.h5", save_freq=5)]

In [92]:
# Train the model
nn.fit(X_train_scaled, y_train, validation_data=(X_test_scaled,y_test), epochs=50, callbacks=cb)

Epoch 1/50
 15/804 [..............................] - ETA: 9s - loss: 0.7453 - accuracy: 0.5417 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


804/804 [==============================] - 13s 14ms/step - loss: 0.5760 - accuracy: 0.7130 - val_loss: 0.5598 - val_accuracy: 0.7233 - lr: 0.0010
Epoch 2/50
804/804 [==============================] - 11s 13ms/step - loss: 0.5588 - accuracy: 0.7247 - val_loss: 0.5542 - val_accuracy: 0.7278 - lr: 0.0010
Epoch 3/50
804/804 [==============================] - 11s 14ms/step - loss: 0.5568 - accuracy: 0.7265 - val_loss: 0.5534 - val_accuracy: 0.7269 - lr: 0.0010
Epoch 4/50
804/804 [==============================] - 10s 12ms/step - loss: 0.5547 - accuracy: 0.7282 - val_loss: 0.5521 - val_accuracy: 0.7315 - lr: 0.0010
Epoch 5/50
804/804 [==============================] - 11s 13ms/step - loss: 0.5532 - accuracy: 0.7287 - val_loss: 0.5524 - val_accuracy: 0.7282 - lr: 0.0010
Epoch 6/50
804/804 [==============================] - 10s 13ms/step - loss: 0.5523 - accuracy: 0.7283 - val_loss: 0.5516 - val_accuracy: 0.7259 - lr: 0.0010
Epoch 7/50
804/804 [==============================] - 11s 13ms/step -

In [93]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5482 - accuracy: 0.7312 - 672ms/epoch - 3ms/step
Loss: 0.5481560230255127, Accuracy: 0.731195330619812


In [95]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')